In [1]:
import chess
import numpy as np
import tensorflow as tf
import random
import math

In [2]:
def board_to_cnn_input(board):
    planes = np.zeros((8, 8, 19), dtype=np.float32)
    piece_map = board.piece_map()

    piece_to_plane = {
        chess.PAWN: 0, chess.KNIGHT: 1, chess.BISHOP: 2,
        chess.ROOK: 3, chess.QUEEN: 4, chess.KING: 5,
    }

    for square, piece in piece_map.items():
        rank, file = divmod(square, 8)
        idx = piece_to_plane[piece.piece_type] + (6 if piece.color == chess.BLACK else 0)
        planes[rank, file, idx] = 1

    planes[:, :, 12] = int(board.turn == chess.WHITE)
    planes[:, :, 13] = int(board.has_kingside_castling_rights(chess.WHITE))
    planes[:, :, 14] = int(board.has_queenside_castling_rights(chess.WHITE))
    planes[:, :, 15] = int(board.has_kingside_castling_rights(chess.BLACK))
    planes[:, :, 16] = int(board.has_queenside_castling_rights(chess.BLACK))

    if board.ep_square:
        ep_rank, ep_file = divmod(board.ep_square, 8)
        planes[ep_rank, ep_file, 17] = 1

    planes[:, :, 18] = board.halfmove_clock / 100
    return planes


In [3]:
class MCTSNode:
    def __init__(self, parent, move, board):
        self.parent = parent
        self.move = move
        self.board = board
        self.children = []
        self.visits = 0
        self.value = 0

    def is_fully_expanded(self):
        return len(self.children) == len(list(self.board.legal_moves))

    def best_child(self, exploration_weight=1.414):
        if not self.children:
            return None
        
        scores = []
        for child in self.children:
        # Q-value (exploitation term)
            exploitation = child.value / (child.visits + 1e-8)
        
        # UCB exploration term
            exploration = exploration_weight * math.sqrt(
                math.log(self.visits + 1) / (child.visits + 1e-8)
            )
        
        # Flip score for black's moves
            if not self.board.turn:
                exploitation = -exploitation
            
            scores.append(exploitation + exploration)
    
        return self.children[np.argmax(scores)]

    def select_child(self):
        if len(self.children) == 0:
            return self
        return self.best_child()

    def expand(self):
        for move in self.board.legal_moves:
            new_board = self.board.copy()
            new_board.push(move)
            child_node = MCTSNode(self, move, new_board)
            self.children.append(child_node)

    def simulate(self, eval_cnn_v2):
        cnn_input = board_to_cnn_input(self.board)
        eval_score = eval_cnn_v2.predict(np.expand_dims(cnn_input, axis=0))[0][0]
        return eval_score

    def backpropagate(self, result):
        self.visits += 1
    
    # For chess, we need to flip the evaluation based on player
        actual_result = result if self.board.turn else -result
    
    # Update running average
        self.value += (actual_result - self.value) / self.visits
    
        if self.parent:
            self.parent.backpropagate(result)  # Pass original result to parent

class MCTS:
    def __init__(self, model, root_board):
        self.model = model
        self.root = MCTSNode(None, None, root_board)

    def search(self, iterations=1000):
        for _ in range(iterations):
            node = self.root
            # Selection
            while node.is_fully_expanded() and node.children:
                node = node.select_child()
            # Expansion
            if not node.is_fully_expanded():
                node.expand()
                node = node.best_child()
            # Simulation
            result = node.simulate(self.model)
            # Backpropagation
            node.backpropagate(result)
        return self.best_move()

    def best_move(self):
        best_node = self.root.best_child()
        return best_node.move


In [4]:
# Load a model from a .h5 file
eval_cnn_v5 = tf.keras.models.load_model(r'D:\Aarti\CNN_model\eval_cnn_v5.keras')


In [7]:
# Initialize the board and the model (assuming eval_cnn_v2 is loaded)
# board = chess.Board()

# Initialize MCTS with your eval model and the initial board
mcts = MCTS(eval_cnn_v5, board)

# Perform MCTS to get the best move
best_move = mcts.search(iterations=100)
print(f"Best Move: {best_move}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━

In [ ]:
move = "g5f7"  # Knight moves from g1 to f3

# Convert to a chess.Move object
move_obj = chess.Move.from_uci(move)

# Apply the best move to the board
board.push(best_move)
print(board)


In [47]:
board = chess.Board()
while not board.is_game_over():
    mcts = MCTS(eval_cnn_v5, board)
    best_move = mcts.search(iterations=10)
    print(f"Move: {best_move}")
    board.push(best_move)
    print(board)
    print("\n")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Move: g1h3
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . N
P P P P P P P P
R N B Q K B . R


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Move: g8h6
r n b q k b . r
p p p p p p p p
. . . . . . . n
. . . . . . . .
. . . . . . . .
. . . . 

In [ ]:
import chess
import chess.pgn
import datetime

def play_and_save_game(eval_cnn_v5, iterations=10, filename="game.pgn"):
    # Initialize the board
    board = chess.Board()
    
    # Create a new PGN game object with headers
    game = chess.pgn.Game()
    
    # Add game metadata
    game.headers["Event"] = "Computer Chess Game"
    game.headers["Site"] = "Local Computer"
    game.headers["Date"] = datetime.datetime.now().strftime("%Y.%m.%d")
    game.headers["Round"] = "1"
    game.headers["White"] = "MCTS Engine"
    game.headers["Black"] = "MCTS Engine"
    game.headers["Result"] = "*"  # Will be updated when game ends
    
    # Keep track of moves for the PGN
    node = game
    
    move_number = 1
    try:
        while not board.is_game_over():
            # Get the best move from MCTS
            mcts = MCTS(eval_cnn_v5, board)
            best_move = mcts.search(iterations=iterations)
            
            if best_move is None:
                print(f"No valid move found at position {move_number}")
                break
                
            # Print current move
            print(f"Move {move_number}: {board.san(best_move)}")
            
            # Push the move to the board
            board.push(best_move)
            
            # Add the move to the PGN game
            node = node.add_variation(best_move)
            
            # Print the current board state
            print(board)
            print("\n")
            
            move_number += 1
            
    except Exception as e:
        print(f"Error during game play: {e}")
        
    # Update the game result
    if board.is_checkmate():
        result = "0-1" if board.turn else "1-0"
    elif board.is_stalemate() or board.is_insufficient_material() or board.is_fifty_moves() or board.is_repetition():
        result = "1/2-1/2"
    else:
        result = "*"
    
    game.headers["Result"] = result
    
    # Save the PGN
    try:
        with open(filename, "w", encoding="utf-8") as pgn_file:
            exporter = chess.pgn.FileExporter(pgn_file)
            game.accept(exporter)
        print(f"Game saved successfully to {filename}")
        
        # Also return the game object for further use if needed
        return game
        
    except Exception as e:
        print(f"Error saving game: {e}")
        return None

# Example usage:
if __name__ == "__main__":
    # Assuming eval_cnn_v2 is your evaluation model
    game = play_and_save_game(
        eval_cnn_v5=eval_cnn_v5,
        iterations=50,
        filename="mcts_game.pgn"
    )

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86

In [128]:
import math
import numpy as np
import chess
import chess.engine

class MCTSNode:
    def __init__(self, parent, move, board):
        self.parent = parent
        self.move = move
        self.board = board
        self.children = []
        self.visits = 0
        self.value = 0

    def is_fully_expanded(self):
        return len(self.children) == len(list(self.board.legal_moves))

    def best_child(self, exploration_weight=1.414):
        if not self.children:
            return None
        
        scores = []
        for child in self.children:
            # Q-value (exploitation term)
            exploitation = child.value / (child.visits + 1e-8)
        
            # UCB exploration term
            exploration = exploration_weight * math.sqrt(
                math.log(self.visits + 1) / (child.visits + 1e-8)
            )
        
            # Flip score for black's moves
            if not self.board.turn:
                exploitation = -exploitation
            
            scores.append(exploitation + exploration)
    
        return self.children[np.argmax(scores)]

    def select_child(self):
        if len(self.children) == 0:
            return self
        return self.best_child()

    def expand(self):
        # Expand only one child at a time (lazy expansion)
        for move in self.board.legal_moves:
            new_board = self.board.copy()
            new_board.push(move)
            child_node = MCTSNode(self, move, new_board)
            self.children.append(child_node)
            return child_node  # Expand one child at a time

    def simulate(self, stockfish_engine):
        # Terminal state check (e.g., checkmate, stalemate)
        if self.board.is_game_over():
            if self.board.result() == "1-0":
                return 1  # White wins
            elif self.board.result() == "0-1":
                return -1  # Black wins
            else:
                return 0  # Draw
        
        # Otherwise, use Stockfish to evaluate the position
        stockfish_engine.position(self.board)
        evaluation = stockfish_engine.go(movetime=2000)  # Evaluate for 2 seconds (adjust if needed)
        score = evaluation["score"].relative.score(mate_score=10000)  # Normalize for mate scores
        return score

    def backpropagate(self, result):
        self.visits += 1
    
        # Flip result for Black's turn (since Stockfish might be trained for White's perspective)
        actual_result = result if self.board.turn else -result
    
        # Update running average
        self.value += (actual_result - self.value) / self.visits
    
        if self.parent:
            self.parent.backpropagate(result)  # Pass original result to parent


class MCTS:
    def __init__(self, stockfish_path, root_board):
        self.stockfish_path = stockfish_path
        self.root = MCTSNode(None, None, root_board)
        self.stockfish_engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)

    def search(self, iterations=1000):
        for _ in range(iterations):
            node = self.root
            # Selection
            while node.is_fully_expanded() and node.children:
                node = node.select_child()
            # Expansion (add only one child at a time)
            if not node.is_fully_expanded():
                node.expand()
                node = node.best_child()
            # Simulation
            result = node.simulate(self.stockfish_engine)
            # Backpropagation
            node.backpropagate(result)
        return self.best_move()

    def best_move(self):
        best_node = self.root.best_child()
        return best_node.move

    def close(self):
        self.stockfish_engine.quit()


# Example Usage:
if __name__ == "__main__":
    # Initialize chess board
    board = chess.Board()

    # Path to Stockfish engine (download the executable from the Stockfish website)
    stockfish_path = "D:\\Stockfish\\stockfish-windows-x86-64-sse41-popcnt\\stockfish\\stockfish-windows-x86-64-sse41-popcnt"  # Update this to your Stockfish binary path

    # Initialize MCTS with Stockfish
    mcts = MCTS(stockfish_path, board)

    # Perform MCTS search for 1000 iterations
    best_move = mcts.search(10)
    print("Best move found:", best_move)

    # Close the Stockfish engine
    mcts.close()


NotImplementedError: 